# EasyTPP - Getting Started Guide

This notebook presents the main features of the **EasyTPP** (Easy Temporal Point Processes) library with practical examples.

## 🎯 Notebook Objectives

- Understand the basic concepts of temporal point processes
- Learn to configure and train models
- Explore the different types of data and available models
- Visualize and analyze results

## 📚 Table of Contents

1. [Environment Setup](#1-configuration)
2. [Basic Concepts](#2-concepts)
3. [Data Loading and Preparation](#3-donnees)
4. [Model Configuration and Training](#4-entrainement)
5. [Evaluation and Metrics](#5-evaluation)
6. [Advanced Examples](#6-avances)

## 1. Environment Setup {#1-configuration}

Let's start by importing the necessary modules and setting up the environment.

In [27]:
import sys
from pathlib import Path

# Add the project root directory to PYTHONPATH
ROOT = Path().absolute().parent.parent

CONFIGS = ROOT / "configs" / "test_runner_config.yaml"

# EasyTPP imports
from easy_tpp.config_factory import RunnerConfig
from easy_tpp.utils.yaml_config_utils import parse_runner_yaml_config
from easy_tpp.runners import Runner

print("✅ EasyTPP imported successfully!")
print(f"📁 Project directory: {ROOT}")

✅ EasyTPP imported successfully!
📁 Project directory: c:\Users\enzo.cAo\Documents\Projects\finance\projet_recherche\New_LTPP


## 2. Basic Concepts {#2-concepts}

### What is a Temporal Point Process?

A **Temporal Point Process** (TPP) is a sequence of events that occur over time. Each event is characterized by:

- **Occurrence time**: When the event happens
- **Event type**: What category of event (optional)

### Application examples:

- 🏥 **Medical**: Patient arrivals at a hospital
- 💰 **Finance**: Stock market transactions
- 🌍 **Geophysics**: Earthquakes
- 📱 **Social Networks**: User posts

### Models available in EasyTPP:

- **NHP** (Neural Hawkes Process): Hawkes processes with neural networks
- **THP** (Transformer Hawkes Process): Based on Transformer architecture
- **RMTPP** (Recurrent Marked Temporal Point Process): Based on RNNs
- **AttNHP** (Attentive Neural Hawkes Process): With attention mechanism

## 3. Data Loading and Preparation {#3-donnees}

EasyTPP supports multiple data formats. Let's see how to load and prepare data.

In [2]:
from easy_tpp.config_factory import DataConfig
from easy_tpp.config_factory.data_config import DataLoadingSpecsConfig, TokenizerConfig
from easy_tpp.data.preprocess import TPPDataModule

In [3]:
# Data configuration with proper nested structure
data_config = DataConfig(
    test_dir="NzoCs/test_dataset",                  # Directory for test data
    valid_dir="NzoCs/test_dataset",                 # Directory for validation data
    train_dir="NzoCs/test_dataset",            # Source directory for data
    dataset_id="test",                          # Dataset to use
    data_format="json",                       # Data format (pickle, json, csv)
    data_loading_specs=DataLoadingSpecsConfig(
        batch_size=32,                          # Batch size
        num_workers=1,                          # Number of workers for data loading
        shuffle=True                            # Shuffle data
    ),
    data_specs=TokenizerConfig(
        num_event_types=2,                      # Number of event types
        padding_side='left',                    # Padding side
        truncation_side='left'                  # Truncation side
    )
)

print("📊 Data configuration created:")
print(f"   Dataset: {data_config.dataset_id}")
print(f"   Format: {data_config.data_format}")
print(f"   Event types: {data_config.data_specs.num_event_types}")
print(f"   Batch size: {data_config.data_loading_specs.batch_size}")
print(f"   Number of workers: {data_config.data_loading_specs.num_workers}")
print(f"   Padding side: {data_config.data_specs.padding_side}")

📊 Data configuration created:
   Dataset: test
   Format: json
   Event types: 2
   Batch size: 32
   Number of workers: 1
   Padding side: left


In [5]:
# Alternative: Create DataConfig using from_dict (simpler approach)
data_config_dict = {
    "train_dir": "NzoCs/test_dataset",
    "valid_dir": "NzoCs/test_dataset",
    "test_dir": "NzoCs/test_dataset",
    "dataset_id": "test",
    "data_format": "json",
    "data_loading_specs": {
        "batch_size": 32,
        "num_workers": 1,
        "shuffle": True
    },
    "data_specs": {
        "num_event_types": 2,
        "padding_side": "left",
        "truncation_side": "left"
    }
}

# Create DataConfig from dictionary
data_config_alt = DataConfig.from_dict(data_config_dict)

print("📊 Alternative DataConfig created from dictionary:")
print(f"   Dataset: {data_config_alt.dataset_id}")
print(f"   Format: {data_config_alt.data_format}")
print(f"   Event types: {data_config_alt.data_specs.num_event_types}")
print(f"   Batch size: {data_config_alt.data_loading_specs.batch_size}")

📊 Alternative DataConfig created from dictionary:
   Dataset: test
   Format: json
   Event types: 2
   Batch size: 32


In [6]:
# Create data module
datamodule = TPPDataModule(data_config_alt)
datamodule.setup(stage='fit')  # Setup for training and validation

# Get data loaders
train_loader = datamodule.train_dataloader()
val_loader = datamodule.val_dataloader()

print("✅ Data loaders created successfully!")
print(f"   📈 Train loader: {len(train_loader)} batches")
print(f"   📊 Validation loader: {len(val_loader)} batches")

2025-09-18 02:53:53,684 - data_loader.py[pid:7980;line:140:setup] - INFO: Setting up data for stage: fit


c:\Users\enzo.cAo\Documents\Projects\finance\projet_recherche\New_LTPP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-09-18 02:54:11,427 - data_loader.py[pid:7980;line:149:setup] - INFO: Train dataset created with 6 sequences
2025-09-18 02:54:20,206 - data_loader.py[pid:7980;line:158:setup] - INFO: Validation dataset created with 2 sequences
✅ Data loaders created successfully!
   📈 Train loader: 1 batches
   📊 Validation loader: 1 batches


### Data Inspection

Let's use the Visualizer to analyze the data distribution.

In [28]:
from easy_tpp.data.preprocess.visualizer import Visualizer

# Create the visualizer
visualizer = Visualizer(
    data_module=datamodule,
    split="train",
    save_dir="./analysis_plots"
)

# Generate visualizations
visualizer.show_all_distributions()
visualizer.delta_times_distribution()
visualizer.event_type_distribution()

print("📈 Analysis plots generated!")
print("   Check the './analysis_plots' folder for saved graphs")

Generating visualization plots...
Inter-event time distribution plot saved to ./analysis_plots\inter_event_time_dist.png
Event type distribution plot saved to ./analysis_plots\event_type_dist.png
Sequence length distribution plot saved to ./analysis_plots\sequence_length_dist.png
All plots generated successfully!
Inter-event time distribution plot saved to ./analysis_plots\inter_event_time_dist.png
Event type distribution plot saved to ./analysis_plots\event_type_dist.png
📈 Analysis plots generated!
   Check the './analysis_plots' folder for saved graphs


## 4. Model Configuration and Training {#4-entrainement}

Now, let's configure and train a Neural Hawkes Process (NHP) model.

In [29]:
# Load configuration from YAML file
runner_dict = parse_runner_yaml_config(str(CONFIGS), "NHP", "test")

# Create configuration
config = RunnerConfig.from_dict(runner_dict)

print("⚙️ Model configuration:")
print(f"   🧠 Model: NHP")
print(f"   📊 Dataset: test")

⚙️ Model configuration:
   🧠 Model: NHP
   📊 Dataset: test


In [30]:
# Create runner and start training
runner = Runner(config=config, output_dir="./training_results")

print("🚀 Starting training...")
print("   This may take a few minutes depending on your configuration.")

# Train the model
runner.run(phase="train")

print("✅ Training completed!")

2025-09-18 03:16:52,747 - runner.py[pid:7980;line:39:__init__] - CRITICAL: Runner initialized for model: NHP on dataset: test
🚀 Starting training...
   This may take a few minutes depending on your configuration.
2025-09-18 03:16:52,757 - runner.py[pid:7980;line:129:run] - INFO: Runner executing phases: ['train']
2025-09-18 03:16:52,762 - runner.py[pid:7980;line:72:train] - INFO: === TRAINING PHASE ===
2025-09-18 03:16:52,784 - model_runner.py[pid:7980;line:116:__init__] - INFO: No valid checkpoint found. Starting from scratch.
2025-09-18 03:16:52,794 - model_runner.py[pid:7980;line:221:train] - INFO: --- Starting Training for Model : NHP on dataset : test ---


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


2025-09-18 03:16:53,036 - data_loader.py[pid:7980;line:140:setup] - INFO: Setting up data for stage: fit
2025-09-18 03:17:02,152 - data_loader.py[pid:7980;line:149:setup] - INFO: Train dataset created with 6 sequences
2025-09-18 03:17:12,738 - data_loader.py[pid:7980;line:158:setup] - INFO: Validation dataset created with 2 sequences



  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | layer_type_emb  | Embedding        | 192    | train
1 | rnn_cell        | ContTimeLSTMCell | 57.8 K | train
2 | layer_intensity | Sequential       | 132    | train
-------------------------------------------------------------
58.1 K    Trainable params
0         Non-trainable params
58.1 K    Total params
0.232     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\enzo.cAo\Documents\Projects\finance\projet_recherche\New_LTPP\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


c:\Users\enzo.cAo\Documents\Projects\finance\projet_recherche\New_LTPP\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
c:\Users\enzo.cAo\Documents\Projects\finance\projet_recherche\New_LTPP\.venv\Lib\site-packages\pytorch_lightning\loops\fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=5). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 4: 100%|██████████| 1/1 [00:00<00:00,  3.64it/s, v_num=0, train_loss=2.460]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 1/1 [00:00<00:00,  3.56it/s, v_num=0, train_loss=2.460]
✅ Training completed!


## 5. Evaluation and Metrics {#5-evaluation}

Let's now evaluate the performance of the trained model.

In [31]:
# Evaluation on test dataset
print("🧪 Evaluating model on test dataset...")

test_results = runner.run(phase="test")

print("📊 Evaluation results:")
if hasattr(runner, 'test_metrics'):
    for metric_name, value in runner.test_metrics.items():
        print(f"   {metric_name}: {value:.4f}")
else:
    print("✅ Evaluation completed - check logs for detailed metrics")

🧪 Evaluating model on test dataset...
2025-09-18 03:18:22,179 - runner.py[pid:7980;line:129:run] - INFO: Runner executing phases: ['test']
2025-09-18 03:18:22,180 - runner.py[pid:7980;line:85:test] - CRITICAL: === TESTING PHASE ===
2025-09-18 03:18:22,184 - model_runner.py[pid:7980;line:103:__init__] - INFO: Checkpoint found: loading from ./training_results\last.ckpt
2025-09-18 03:18:22,194 - model_runner.py[pid:7980;line:114:__init__] - INFO: Loading model from checkpoint: ./training_results\last.ckpt.
2025-09-18 03:18:22,197 - model_runner.py[pid:7980;line:245:test] - INFO: --- Starting Testing for Model : NHP on dataset : test ---


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


2025-09-18 03:18:22,233 - data_loader.py[pid:7980;line:140:setup] - INFO: Setting up data for stage: test


Restoring states from the checkpoint path at ./training_results\last.ckpt
Loaded model weights from the checkpoint at ./training_results\last.ckpt
c:\Users\enzo.cAo\Documents\Projects\finance\projet_recherche\New_LTPP\.venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00,  7.47it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       cross_entropy       │    0.49844682216644287    │
│       macro_f1score       │     81.37931060791016     │
│         precision         │     81.37931060791016     │
│          recall           │     81.37931060791016     │
│         test_loss         │    2.0783979892730713     │
│         time_mae          │    1.7528632879257202     │
│         time_rmse         │    3.0204854011535645     │
│       type_accuracy       │     81.48148345947266     │
└───────────────────────────┴───────────────────────────┘

2025-09-18 03:18:59,164 - model_runner.py[pid:7980;line:269:test] - INFO: Test results saved to ./training_results\test_results.json
📊 Evaluation results:
✅ Evaluation completed - check logs for detailed metrics
📊 Evaluation results:
✅ Evaluation completed - check logs for detailed metrics


### Comparison with Baselines

Let's compare our model with simple baselines.

In [17]:
from easy_tpp.evaluation.benchmarks.mean_bench import MeanInterTimeBenchmark
from easy_tpp.evaluation.benchmarks.sample_distrib_mark_bench import MarkDistributionBenchmark

# Baseline benchmark: mean prediction
mean_benchmark = MeanInterTimeBenchmark(
    data_config=data_config_alt,
    experiment_id="mean_baseline",
    save_dir="./benchmark_results"
)

print("📊 Baseline benchmark (mean):")
mean_results = mean_benchmark.evaluate()
print(f"   Results: {mean_results}")

# Type distribution benchmark
mark_benchmark = MarkDistributionBenchmark(
    data_config=data_config_alt,
    experiment_id="mark_baseline",
    save_dir="./benchmark_results"
)

print("\n📊 Type distribution benchmark:")
mark_results = mark_benchmark.evaluate()
print(f"   Results: {mark_results}")

2025-09-18 03:01:33,627 - data_loader.py[pid:7980;line:140:setup] - INFO: Setting up data for stage: test


📊 Baseline benchmark (mean):
2025-09-18 03:01:37,917 - base_bench.py[pid:7980;line:147:evaluate] - INFO: Starting mean_inter_time benchmark evaluation...
2025-09-18 03:01:37,918 - mean_bench.py[pid:7980;line:51:_prepare_benchmark] - INFO: Computing mean inter-time from training data...
2025-09-18 03:01:37,918 - mean_bench.py[pid:7980;line:51:_prepare_benchmark] - INFO: Computing mean inter-time from training data...
2025-09-18 03:02:08,479 - mean_bench.py[pid:7980;line:74:_prepare_benchmark] - INFO: Computed mean inter-time: 1.506245
2025-09-18 03:02:37,831 - base_bench.py[pid:7980;line:366:_save_results] - INFO: Results saved to: ./benchmark_results\mean_baseline\mean_inter_time_results.json
2025-09-18 03:02:37,833 - base_bench.py[pid:7980;line:375:_log_summary] - INFO: mean_inter_time benchmark completed successfully!
2025-09-18 03:02:37,836 - base_bench.py[pid:7980;line:381:_log_summary] - INFO: Time RMSE: 2.772637
2025-09-18 03:02:37,838 - base_bench.py[pid:7980;line:383:_log_summa

## 6. Advanced Examples {#6-avances}

### Synthetic Data Generation

EasyTPP allows generating synthetic data to test models.

In [1]:
from easy_tpp.data.generation import HawkesSimulator

# Hawkes process configuration
params = {
    "mu": [0.1, 0.2],                    # Base intensities
    "alpha": [[0.3, 0.1], [0.2, 0.4]],  # Excitation matrix
    "beta": [[2, 1], [1.5, 3]]          # Decay matrix
}

# Create simulator
simulator = HawkesSimulator(
    mu=params["mu"],
    alpha=params["alpha"],
    beta=params["beta"],
    dim_process=2,
    start_time=0,
    end_time=100
)

print("🎲 Generating synthetic data...")

# Generate and save
simulator.generate_and_save(
    output_dir='./synthetic_data',
    num_simulations=10,
    splits={'train': 0.6, 'test': 0.2, 'dev': 0.2}
)

print("✅ Synthetic data generated in './synthetic_data'")

🎲 Generating synthetic data...
Génération de 10 simulations 2D...


Simulation de 10 processus: 100%|██████████| 10/10 [00:00<00:00, 33.72it/s]

Division des données en ensembles train/test/dev...
Sauvegarde des données...
Toutes les données ont été sauvegardées dans ./synthetic_data
✅ Synthetic data generated in './synthetic_data'


### Multiple Model Comparison

Let's compare the performance of different models on the same dataset.

In [32]:
# List of models to compare
models_to_compare = ['NHP', 'THP', 'RMTPP']
results_comparison = {}

for model_name in models_to_compare:
    print(f"\n🧠 Training model {model_name}...")
    
    try:
        # Configuration for this model
        model_config_dict = parse_runner_yaml_config(str(CONFIGS), model_name, "test")

        config = RunnerConfig.from_dict(model_config_dict)
        runner = Runner(config=config, output_dir=f"./comparison_results/{model_name}")
        
        # Quick training (fewer epochs for demo)
        runner.run(phase="train")
        test_results = runner.run(phase="test")
        
        results_comparison[model_name] = "✅ Success"
        print(f"   ✅ {model_name} trained successfully")
        
    except Exception as e:
        results_comparison[model_name] = f"❌ Error: {str(e)[:50]}..."
        print(f"   ❌ Error with {model_name}: {str(e)[:50]}...")

print("\n📊 Comparison summary:")
for model, result in results_comparison.items():
    print(f"   {model}: {result}")


🧠 Training model NHP...
2025-09-18 03:19:45,632 - runner.py[pid:7980;line:39:__init__] - CRITICAL: Runner initialized for model: NHP on dataset: test
2025-09-18 03:19:45,633 - runner.py[pid:7980;line:129:run] - INFO: Runner executing phases: ['train']
2025-09-18 03:19:45,634 - runner.py[pid:7980;line:72:train] - INFO: === TRAINING PHASE ===
2025-09-18 03:19:45,637 - model_runner.py[pid:7980;line:116:__init__] - INFO: No valid checkpoint found. Starting from scratch.
2025-09-18 03:19:45,638 - model_runner.py[pid:7980;line:221:train] - INFO: --- Starting Training for Model : NHP on dataset : test ---


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


2025-09-18 03:19:45,665 - data_loader.py[pid:7980;line:140:setup] - INFO: Setting up data for stage: fit
2025-09-18 03:19:49,379 - data_loader.py[pid:7980;line:149:setup] - INFO: Train dataset created with 6 sequences
2025-09-18 03:19:54,366 - data_loader.py[pid:7980;line:158:setup] - INFO: Validation dataset created with 2 sequences



  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | layer_type_emb  | Embedding        | 192    | train
1 | rnn_cell        | ContTimeLSTMCell | 57.8 K | train
2 | layer_intensity | Sequential       | 132    | train
-------------------------------------------------------------
58.1 K    Trainable params
0         Non-trainable params
58.1 K    Total params
0.232     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


SystemExit: 1

c:\Users\enzo.cAo\Documents\Projects\finance\projet_recherche\New_LTPP\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 6. Prediction Phase and Distribution Analysis

**Why the prediction phase is crucial:**

Temporal Point Process (TPP) models don't just serve to calculate performance metrics - their true value lies in their ability to **predict and simulate** new events. These predictions enable:

1. **Distribution comparisons** - Analyze whether the model captures temporal patterns well
2. **Realistic benchmarks** - Compare model simulations to real data  
3. **Qualitative validation** - Visualize differences between predictions and reality
4. **Practical applications** - Generate future scenarios for decision-making

### 6.1 Complete Pipeline with Predictions

In [33]:
# Complete example: train → test → predict
print("🔄 Complete pipeline with predictions...")

# Configuration
config_dict = parse_runner_yaml_config(
    yaml_path=CONFIGS,
    experiment_id="NHP", 
    dataset_id="test"
)
config = RunnerConfig.from_dict(config_dict)

# Runner
runner = Runner(config=config, output_dir="./prediction_analysis")

# Phase 1: Training
print("📚 1. Training the model...")
runner.run(phase="train")

# Phase 2: Test/Evaluation  
print("🧪 2. Performance evaluation...")
runner.run(phase="test")

# Phase 3: Predictions and comparisons (CRUCIAL!)
print("🔮 3. Generating predictions and distribution comparisons...")
runner.run(phase="predict")

print("✅ Complete pipeline finished!")
print("📊 Results available in:")
print("   - Performance metrics")
print("   - Model simulations") 
print("   - Distribution comparisons")
print("   - Analysis graphs")

🔄 Complete pipeline with predictions...
2025-09-18 03:20:21,406 - runner.py[pid:7980;line:39:__init__] - CRITICAL: Runner initialized for model: NHP on dataset: test
📚 1. Training the model...
2025-09-18 03:20:21,412 - runner.py[pid:7980;line:129:run] - INFO: Runner executing phases: ['train']
2025-09-18 03:20:21,419 - runner.py[pid:7980;line:72:train] - INFO: === TRAINING PHASE ===
2025-09-18 03:20:21,437 - model_runner.py[pid:7980;line:116:__init__] - INFO: No valid checkpoint found. Starting from scratch.
2025-09-18 03:20:21,440 - model_runner.py[pid:7980;line:221:train] - INFO: --- Starting Training for Model : NHP on dataset : test ---


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


2025-09-18 03:20:21,592 - data_loader.py[pid:7980;line:140:setup] - INFO: Setting up data for stage: fit
2025-09-18 03:20:30,059 - data_loader.py[pid:7980;line:149:setup] - INFO: Train dataset created with 6 sequences
2025-09-18 03:20:38,542 - data_loader.py[pid:7980;line:158:setup] - INFO: Validation dataset created with 2 sequences



  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | layer_type_emb  | Embedding        | 192    | train
1 | rnn_cell        | ContTimeLSTMCell | 57.8 K | train
2 | layer_intensity | Sequential       | 132    | train
-------------------------------------------------------------
58.1 K    Trainable params
0         Non-trainable params
58.1 K    Total params
0.232     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Epoch 4: 100%|██████████| 1/1 [00:00<00:00,  3.95it/s, v_num=3, train_loss=2.400]

...

Predicting DataLoader 0: 100%|██████████| 1/1 [00:38<00:00,  0.03it/s]


Formatting sequences: 100%|██████████| 2/2 [00:00<00:00, 1152.44it/s]

Data successfully saved to .\distributions_comparisons\simulations.json
2025-09-18 03:23:46,469 - comparator.py[pid:7980;line:142:create_comparator] - INFO: Using TPPDatasetExtractor for optimized data extraction
2025-09-18 03:23:46,476 - comparator.py[pid:7980;line:69:run_comprehensive_evaluation] - INFO: Starting comprehensive temporal point process evaluation...
2025-09-18 03:23:46,482 - data_extractors.py[pid:7980;line:69:_extract_all_data] - INFO: Extracting ground truth data from TPPDataset with 2 sequences...
2025-09-18 03:23:46,496 - data_extractors.py[pid:7980;line:118:_extract_all_data] - INFO: Successfully processed 2/2 sequences, extracted 56 events
2025-09-18 03:23:46,501 - data_extractors.py[pid:7980;line:277:_extract_all_data] - INFO: Processing simulation data...


2025-09-18 03:23:49,798 - distribution_analyzer.py[pid:7980;line:119:plot_density_comparison] - INFO: Density comparison plot successfully saved to .\distributions_comparisons\comparison_inter_event_time_dist.png
2025-09-18 03:23:53,015 - plot_generators.py[pid:7980;line:165:generate_plot] - INFO: Event type distribution comparison plot saved to .\distributions_comparisons\comparison_event_type_dist.png
2025-09-18 03:23:55,458 - plot_generators.py[pid:7980;line:257:generate_plot] - INFO: Sequence length distribution comparison plot saved to .\distributions_comparisons\comparison_sequence_length_dist.png
2025-09-18 03:24:00,065 - plot_generators.py[pid:7980;line:342:generate_plot] - INFO: Cross-correlation comparison plot saved to .\distributions_comparisons\comparison_cross_correlation_moments.png
2025-09-18 03:24:00,067 - comparator.py[pid:7980;line:97:run_comprehensive_evaluation] - INFO: Comprehensive evaluation completed successfully
2025-09-18 03:24:00,070 - model_runner.py[pid:79

RuntimeError: Inference tensors cannot be saved for backward. To work around you can make a clone to get a normal tensor and use it in autograd.

### 6.2 Simplified Alternative: Single Command

If you want the complete pipeline all at once:

In [ ]:
# Ultra-simple version: everything in one command
runner = Runner(config=config, output_dir="./complete_pipeline")

# Automatically executes: train → test → predict
runner.run(phase="all")

print("🎉 Complete pipeline executed with phase='all'!")
print("💡 This command is equivalent to the 3 separate phases above")

**🎯 Main objective:** Verify that the model has learned the correct temporal distributions.

**📊 What the `predict` phase generates:**
- **Event simulations** based on the trained model
- **Visual comparisons** between real and simulated data
- **Statistical analyses** of temporal distributions
- **Prediction quality metrics**

**⚠️ Crucial point:** Without the prediction phase, you only have numerical metrics. With predictions, you can **see** if your model truly understands the temporal dynamics of your data.

In [26]:
import shutil
import os

folders_to_remove = [
    "analysis_plots",
    "training_results",
    "benchmark_results",
    "synthetic_data",
    "comparison_results",
    "prediction_analysis",
    "complete_pipeline",
    "lightning_logs",
    "checkpoints"
]

for folder in folders_to_remove:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Deleted: {folder}")
    else:
        print(f"Not found: {folder}")

Not found: analysis_plots
Not found: training_results
Not found: benchmark_results
Not found: synthetic_data
Not found: comparison_results
Not found: prediction_analysis
Not found: complete_pipeline
Deleted: lightning_logs
Deleted: checkpoints
